In [2]:
# DO NOT REMOVE!
import os

import numpy as np
import matplotlib.pyplot as plt

import torch

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import MNIST

In [4]:
# D is gehardcode, D = 28. Verder weet ik niet of deze nets werken met trainen, maar
# de shapes kloppen in elk geval wel.
# inspiratie: https://towardsdatascience.com/building-a-convolutional-vae-in-pytorch-a0f54c947f71
L = 4
encoder = nn.Sequential(nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1), nn.ReLU(), # size: batch, 1, 28, 28, 
                        nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1), nn.ReLU(),  nn.Flatten(1)
                       ,nn.Linear(32*28*28, 2 * L))

decoder = nn.Sequential(nn.Linear(L, 32*28*28), nn.ReLU(), # output shape: (batch, 32*28*28)
                        nn.Unflatten(1, (32, 28, 28)),  # output shape: (batch, 32, 28, 28)
                        nn.ConvTranspose2d(32, 64, kernel_size = 3, padding = 1), nn.ReLU(), #output (batch, 64, 28, 28)
                        nn.ConvTranspose2d(64, 128, kernel_size = 3, padding = 1), nn.ReLU(),#output (batch, 128, 28, 28)
                        nn.ConvTranspose2d(128, 256, kernel_size = 3, padding = 1), nn.ReLU(), #output (batch, 256, 28, 28)
                        nn.Flatten(2), nn.Softmax(dim = -1)) # output (batch, 256, 28*28)


num_vals = 256 
L = 4 # number of latent variables
batch = 2
images = torch.randn(2, 1, 28, 28)
Z = torch.randn(batch, L)

encoder(images).shape # test if encoder works
p= decoder(Z).transpose(1,2).reshape(-1, num_vals) # test if decoder works and reshape to (b*28*28)
p.shape
x_hat = torch.multinomial(p, num_samples = 1).view(batch, 28,28) # sample using probabilities p and torch.multinomial
